In [1]:
from sklearn.neighbors import KNeighborsClassifier

from sklearn.preprocessing import scale

from sklearn.learning_curve import learning_curve
from skimage import draw, data, io, segmentation, color, exposure
from skimage.measure import regionprops
from skimage.filters import threshold_otsu
from skimage.transform import resize 
from skimage.transform import warp 
from PIL import Image

import pickle
import pandas as pd
import numpy as np
from pprint import pprint
from PIL import Image
from gtts import gTTS
import os

/home/dmm/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/dmm/anaconda3/lib/python3.6/site-packages/sklearn/learning_curve.py:22: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
import math

def img_round(x, base=75):

    return (base * math.floor(float(x)/base))
vround = np.vectorize(img_round) 

def get_img(i, size):
    img = Image.open('/home/dmm/ocrfiless/train/'+ str(i+1) + '.bmp')
    img = img.convert("L")
    img = img.resize((size,size))
    image = np.asarray(img)
    image.setflags(write=True)
    return image

def nudge_dataset(X, Y, size):
    direction_vectors = [
        [[0, 1, 0],
         [0, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 0, 0],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [0, 0, 0],
         [0, 1, 0]]]

    shift = lambda x, w: convolve(x.reshape((size, size)), mode='constant',
                                  weights=w).ravel()
    X = np.concatenate([X] +
                       [np.apply_along_axis(shift, 1, X, vector)
                        for vector in direction_vectors])
    Y = np.concatenate([Y for _ in range(5)], axis=0)
    return X, Y

def show_img(img):
    width = 5.0
    height = img.shape[0]*width/img.shape[1]
    f = plt.figure(figsize=(width, height))
    plt.imshow(img)

In [3]:
df = pd.read_csv('/home/dmm/ocrfiless/a.csv', header=0)
raw_y = np.asarray(df['Class'])
raw_x = np.asarray([get_img(i, 50) for i in df.index]).astype(float)
x = np.asarray([i.ravel() for i in raw_x])
y = raw_y
print (x.shape, y.shape)

(10141, 2500) (10141,)


In [4]:
def get_test_img(i):
    img = Image.open('/home/dmm/braille_cells/bmpImg/'+ str(i) + '.bmp')
    img = img.convert("L")
    img = img.resize((50,50))
    image = np.asarray(img)
    image.setflags(write=True)

    return image

In [5]:
from sklearn.cross_validation import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(x, y)

X_train, Y_train = nudge_dataset(X_train, Y_train, 50)



In [6]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)
print('DONE')

DONE


In [11]:
from random import randint

random_image = 898
String=""
for i in range(1,random_image):
    char=str(knn.predict([get_test_img(i).ravel()])).strip('[]\'')
    if char=="space":
        char=" "
    print(char,end="")    
    String=String+char 
    

 waldhand  dollege of  engineering is sit uat ed midway bet ween  sangli and  mirai dit ies at   uishrambag*  sangliw    y  t he   wde dammus is lodat ed on about  yii                                                                           y adres of land on sout hern side of                                                                                   y  sangli  u  mirai roadw                                                                                             y                                                                                                                     y    in yaidg* t he dollege made a modest  beginning as  new  engineering                                               y  dollege* wit h a single mrogram leading                                                                             y t o   bw  ew  diuil degreew                                             

' waldhand  dollege of  engineering is sit uat ed midway bet ween  sangli and  mirai dit ies at   uishrambag*  sangliw    y  t he   wde dammus is lodat ed on about  yii                                                                           y adres of land on sout hern side of                                                                                   y  sangli  u  mirai roadw                                                                                             y                                                                                                                     y    in yaidg* t he dollege made a modest  beginning as  new  engineering                                               y  dollege* wit h a single mrogram leading                                                                             y t o   bw  ew  diuil degreew                                             '

In [15]:
tts = gTTS(text=String, lang='en')
tts.save("/home/dmm/out.mp3")
os.system("mpg321 /home/dmm/out.mp3")

32512

# 